# InceptionV3 (Brain MRI Dataset)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Conv2D, Input, Dropout, Activation, Dense, MaxPooling2D, Flatten, GlobalAveragePooling2D
from keras.optimizers import Adadelta
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.models import load_model
from keras.applications.inception_v3 import InceptionV3

In [ ]:
img_height, img_width = 150, 150  
batch_size = 64

In [ ]:
train_data_dir = "E:\\Dataset_brain_MRI\\Data\\Data"
validation_data_dir = "E:\\Dataset_brain_MRI\\Data\\Data"

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'  
)

validation_generator = val_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

# InceptionV3 Model

In [ ]:
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(256, 256, 3))
transfer_learning_arch = base_model.output
transfer_learning_arch = Conv2D(512, (3, 3), activation='relu', padding='same')(transfer_learning_arch)
transfer_learning_arch = UpSampling2D(size=(2, 2))(transfer_learning_arch)
transfer_learning_arch = Conv2D(256, (3, 3), activation='relu', padding='same')(transfer_learning_arch)
transfer_learning_arch = UpSampling2D(size=(2, 2))(transfer_learning_arch)
transfer_learning_arch = Conv2D(128, (3, 3), activation='relu', padding='same')(transfer_learning_arch)
transfer_learning_arch = UpSampling2D(size=(2, 2))(transfer_learning_arch)
transfer_learning_arch = Conv2D(64, (3, 3), activation='relu', padding='same')(transfer_learning_arch)
transfer_learning_arch = UpSampling2D(size=(2, 2))(transfer_learning_arch)
output = Conv2D(1, (1, 1), activation='sigmoid', padding='same')(x)
transfer_learning_model = Model(inputs=base_model.input, outputs=output)
transfer_learning_model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
transfer_learning_model.summary()

In [ ]:
for layer in transfer_learning_model.layers[:280]:
    layer.trainable = False
for layer in transfer_learning_model.layers[280:]:
    layer.trainable = True

In [ ]:
history_relu=transfer_learning_model.fit(train_generator, batch_size=64, epochs=20, verbose=1, validation_data=val_generator)

# Model Evaluation

In [ ]:
y_predicted = transfer_learning_model.predict(validation_generator)
y_predicted_labels = [np.argmax(i) for i in y_predicted]
cm = tf.math.confusion_matrix(labels=y_test,predictions=y_predicted_labels)

In [ ]:
plt.figure(figsize = (10,7))
sns.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Truth')

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_predicted_labels))

# Model Saving Using FastAPI and Steamlit

In [ ]:
from fastapi import FastAPI
from fastapi.responses import FileResponse
app = FastAPI()
@app.get("/download-model/")
def download_model():
    file_path = "transfer_learning_model.h5" 
    return FileResponse(file_path, media_type='application/octet-stream', filename="transfer_learning_model.h5")

In [ ]:
import streamlit as st
with open("transfer_learning_model.h5", "rb") as file:
    st.download_button(
        label="Download Model",
        data=file,
        file_name="transfer_learning_model.h5",
        mime='application/octet-stream'
    )